# Polygonal Area Light With Linearly Transformed Cosines

## 1. Polygonal diffusion light with lambert diffuse model

Solved by Johann Heinrich Lambert in 18th century. 

$$
E(p_1, ..., p_n) = \frac 1 {2 \pi} \sum_{i=1}^{n} acos(\langle p_i, p_j \rangle) \langle {\frac {p_i \times p_j} {\| {p_i \times p_j} \|} }, {\begin{bmatrix} 0\\ 0\\ 1\\ \end{bmatrix}} \rangle \quad \textrm{where} \quad j = i + 1
$$


## 2. Polygonal diffusion light for microfacet model

### 2.1 Approximate GGX distribution and Smith shadowing term with linearly transformed cosines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors
import ipywidgets as widgets
from ltc_lut import lut_size, ltc_matrix, ltc_amplitude

In [ ]:
%matplotlib widget

def sampleLut(roughness, costheta):
    uvscale = (lut_size - 1.0) / lut_size
    uvbias = 0.5 / lut_size
    uv = np.array([roughness, costheta]) * uvscale + uvbias
    st = uv * lut_size
    iuv = np.floor(st)
    fuv = st - iuv
    
    a = ltc_matrix[int(iuv[1]), int(iuv[0])]
    b = ltc_matrix[int(iuv[1]), np.minimum(63, int(iuv[0] + 1))]
    c = ltc_matrix[np.minimum(63, int(iuv[1] + 1)), int(iuv[0])]
    d = ltc_matrix[np.minimum(63, int(iuv[1] + 1)), np.minimum(63, int(iuv[0]) + 1)]
    lerp = lambda t, a, b: (1.0 - t) * a + t * b
    M = lerp(fuv[1], lerp(fuv[0], a, b), lerp(fuv[0], c, d))
    M = np.transpose(M)
    return M, np.linalg.inv(M)

def evalLtc(L, M, invM):
    Loriginal = np.dot(invM, L)
    Loriginal = Loriginal / np.linalg.norm(Loriginal)

    L_ = np.dot(M, Loriginal)

    l = np.linalg.norm(L_)
    Jacobian = np.linalg.det(M) / (l*l*l);

    D = np.maximum(0.0, Loriginal[2])
    return D / Jacobian

In [ ]:
fig = plt.figure(figsize=[8, 8])
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
    
def plot(roughness, theta):
    grid_res_x = 64
    grid_res_y = grid_res_x

    # convention of scipy: theta is the azimuthal angle and phi is the polar angle
    phiSeq = np.linspace(0, np.pi, num=grid_res_x)
    thetaSeq = np.linspace(0, 2*np.pi, num=grid_res_y)
    phiSeq, thetaSeq = np.meshgrid(phiSeq, thetaSeq)

    x = np.sin(phiSeq) * np.cos(thetaSeq)
    y = np.sin(phiSeq) * np.sin(thetaSeq)
    z = np.cos(phiSeq)

    # clamped cosine lobe
    dist = np.maximum(z, 0)

    M, invM = sampleLut(roughness, np.cos(theta))
    for row in range(grid_res_y):
        for colum in range(grid_res_x):
            L = [x[row][colum], y[row][colum], z[row][colum]]
            dist[row][colum] = evalLtc(L, M, invM)

    normalized_dist = dist / np.maximum(np.nanmax(dist), 1.0)
    ax.plot_surface(-x, y, z, facecolors=cm.jet(normalized_dist), rcount=grid_res_x, ccount=grid_res_y)
    
a = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.8)
t = widgets.FloatSlider(min=0,max=np.pi/2,step=np.pi/200,value=np.pi/4)
widgets.interact(plot, roughness=a, theta=t)

### 2.2 [Approximate Fresnel term separately](http://blog.selfshadow.com/publications/s2016-advances/s2016_ltc_fresnel.pdf)

### 2.3 Put together

HLSL code
```
    float theta = acos(dot(N, V));
    float2 uv = float2(roughness, theta / (0.5 * M_PIf));
    uv = uv * LUT_SCALE + LUT_BIAS;

    const float4 t = ltcMat.Sample(ltcSamp, coord);
    float3x3 invM = float3x3(
        float3(  1,   0, t.y),
        float3(  0, t.z,   0),
        float3(t.w,   0, t.x)
    );

    float res = ltcEvaluate(N, V, P, invM, points, false);

    /* Apply BRDF scale terms (BRDF magnitude and Schlick Fresnel) */
    const float2 schlick = ltcAmp.Sample(ltcSamp, uv).xy;
    res *= (F0 * schlick.x + (1.0 - F0) * schlick.y);
```

## References

* [Real-Time Polygonal-Light Shading with Linearly Transformed Cosines](https://eheitzresearch.wordpress.com/415-2/)
* [Real-Time Area Lighting: a Journey From Research to Production](http://blog.selfshadow.com/publications/s2016-advances/)
* [Real-Time Line- and Disk-Light Shading with Linearly Transformed Cosines](https://labs.unity.com/article/real-time-line-and-disk-light-shading-linearly-transformed-cosines)
* [paper](LTC/LTC.pdf)
* [slides](LTC/slides.pdf)
* [supplemental: MATLAB](LTC/supplemental_matlab.pdf)
* [WebGL Demo](http://blog.selfshadow.com/sandbox/ltc.html)
* [BRDF fitting code](LTC/fit.zip)